### 출처:
https://keras.io/examples/cifar10_resnet/

In [8]:
# from __future__ import print_function
import keras
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from six.moves import cPickle
from sklearn.model_selection import train_test_split

In [4]:
# Training parameters
batch_size = 32 
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean imporves accuracy
subtract_pixel_mean = True

In [5]:
n = 3
version = 1

if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2
    
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

In [9]:
def load_data():
    # Modify path string. use your path which your dataset is in
    path =  r'C:\Users\strea\Links\baseline_code_final (2)'
    fpath = os.path.join(path, 'train_data')
    
    with open(fpath, 'rb') as f:
        d = cPickle.load(f, encoding='bytes')
    X_train = d['data']
    y_train = d['labels']
    X_train = X_train.reshape(X_train.shape[0], 3, 32, 32) # 3072개 숫자가 한줄로 이어진 배열 하나를 32x32짜리 배열 3개로 만듦
    X_train = X_train.transpose(0, 2, 3, 1)
    y_train = np.reshape(y_train, (len(y_train), 1))
    return X_train, y_train

x_train, y_train = load_data()

In [11]:
# seed 값 설정
seed = 0
np.random.seed(seed)
# tf.set_random_seed(seed)

x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.3, random_state=seed)

In [12]:
# Input image dimensions
input_shape = x_train.shape[1:]

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [15]:
# If subtract pixel mean is enabled - 픽셀 몊균값을 빼줌(normalize)
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean # 왜 train의 평균을 빼는거지....
    
print('x_train shape: ', x_train.shape)
print('y_train shape: ', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape:  (28000, 32, 32, 3)
y_train shape:  (28000, 1)
28000 train samples
12000 test samples


In [16]:
# Convert class vectors to binary class matrics.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [17]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    
    # Arguments
        epoch (int): The number of epochs
        
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *- 1e-1
    print('Learning rate: ', lr)
    return lr

In [19]:
def resnet_layer(inputs,
                num_filters=16,
                kernel_size=3,
                strides=1,
                activation='relu',
                batch_normaliztion=True,
                conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    
    # Arguemnts
        input (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions / kernel= 핵, 선대 책 복습
        strides (int): Conv2D square stride dimensions 몇 칸씩 건너 뛸건지
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool) : conv-bn-activation (True) or bn-activation-conv (False) / conv가 앞에오냐 뒤에오냐 차이
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                 kernel_size=kernel_size,
                 strides=stridses,
                 padding='same',
                 kernel_initializer='he_normal',
                 kernel_regularizer=l2(1e-4))
    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x) # 인자가 주어지면 적용하는 코드 
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x) # 인자가 주어지면 적용하는 코드 
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x) # conv가 앞에오냐 뒤에오냐에 따라 적용 순서를 달리함

In [20]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]
    
    Stacks f 2 x (3 x 3) Conv2D-BN-ReLU
    last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled) by a convolutional layer with strides=2,
    while the number of filters is doubled. # 스트라이드로 레이어는 반으로 줄이고, 필터는 두배로 키움
    Within each stage, the layers have the same number filters and the same number of filters.
    각 단계 내에서, 계층들은 동일한 수의 필터와 동일한 수의 필터를 가지고 있다.-파파고
    각 단계 내에서 레이어는 동일한 수의 필터와 같은 수의 필터를가집니다.-구글번역
    뭐라는거냐...
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M
    
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
        
    # Returns 
        model (Model): Keras model instance
    """
    if(depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])') # 왜지....
    # Start model definition
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)
    
    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    #instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0: # first layer but not first stack ??????????
                strides = 2 # downsample 각 단계 앞부분에 줄인다고 했었는데 그건가봄...
            y = resnet_layer(inputs=x,
                            num_filters=num_filters,   # x를 받아서 필터랑 stride 적용해서 y에 저장
                            strides=strides)
            y = resnet_layer(inputs=y,
                            num_filters=num_filters,   # y를 받아서 필터 적용하고 y에 저장
                            activation=None)
            if stack > 0 and res_block == 0: # first Layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model       # 뭔 소린지 모르겠다...